# M1M3 cell learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LightSource as LS
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable

from lsst_efd_client import EfdClient



In [ ]:
client = EfdClient('summit_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.subplot(1,1,1,aspect=1.0)
plt.xlabel("X position (m)")
plt.ylabel("Y position (m)")
plt.title("M1M3 Actuator positions and type\nHardpoints are approximate", fontsize=18)
types = [['SAA','NA', 'o', 'Z', 'b'], ['DAA','+Y', '^', '+Y','g'], ['DAA','-Y', 'v', '-Y', 'cyan'], \
         ['DAA','+X', '>', '+X', 'r'], ['DAA','-X', '<', '-X', 'r']]
for [type, orient, marker, label, color] in types: 
    xs = []
    ys = []
    for i in range(len(FATABLE)):
        x = FATABLE[i][M1M3FATable.FATABLE_XPOSITION]
        y = FATABLE[i][M1M3FATable.FATABLE_YPOSITION]
        if FATABLE[i][M1M3FATable.FATABLE_TYPE] == type and FATABLE[i][M1M3FATable.FATABLE_ORIENTATION] == orient:
            xs.append(x)
            ys.append(y)
        else:
            continue
    plt.scatter(xs, ys, marker=marker, color=color, s=200, label=label)        

# Now plot approximate hardpoint location
Rhp = 3.1 # Radius in meters
for i in range(6):
    theta = 2.0 * np.pi / 6.0 * float(i)
    if i == 0:
        plt.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', s=200, label='HP')
    else:
        plt.scatter(Rhp * np.cos(theta), Rhp * np.sin(theta), marker='o', color='magenta', s=200, label='_nolegend_')
plt.legend(loc='lower left', fontsize=9)

## The plots below compare the "appliedForces"  to the "forceActuatorData"
### I now understand the the "appliedForces" entry in the EFD is the targte force, and the "forceActuatorData"  is the measured force.

In [ ]:
# Times to start looking at encoder values
start = Time("2023-04-18T16:10:00", scale='tai')
end = Time("2023-04-18T16:15:00", scale='tai')

In [ ]:
forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", "*", start.utc, end.utc)

In [ ]:
forces['zForces99'].plot()

In [ ]:
otherForces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", "*", start.utc, end.utc)

In [ ]:
ax = otherForces['zForce99'].plot()
forces['zForces99'].plot(ax=ax)

In [ ]:
# Times of bump test
start = Time("2023-04-17T10:00:00", scale='utc')
end = Time("2023-04-17T11:15:00", scale='utc')

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
len(bumps)

In [ ]:
bumps.head(20)

In [ ]:
plotStart = (start + TimeDelta(40.0, format='sec')).isot
plotEnd = (start + TimeDelta(160.0, format='sec')).isot
bumps['primaryTest0'].plot(label='P0')
bumps['secondaryTest0'].plot(label='S0')
bumps['primaryTest1'].plot(label='P1')
bumps['secondaryTest1'].plot(label='S1')
bumps['primaryTest2'].plot(label='P2')
bumps['secondaryTest2'].plot(label='S2')
plt.xlim(plotStart, plotEnd)
plt.legend()

In [ ]:
test = Time(bumps['timestamp'].values[0], format='unix_tai', scale='tai')

In [ ]:
test.utc.isot

In [ ]:
unixTAIStart = bumps['timestamp'].values[1]
unixTAIEnd = bumps['timestamp'].values[40]
plt.subplot(3,1,1)
plt.plot(forces['timestamp'].values, forces['zForce0'].values)
plt.plot(forces['timestamp'].values, forces['yForce0'].values)
plt.xlim(unixTAIStart, unixTAIEnd)
plt.subplot(3,1,2)
plt.plot(forces['timestamp'].values, forces['zForce1'].values)
plt.plot(forces['timestamp'].values, forces['yForce1'].values)
plt.xlim(unixTAIStart, unixTAIEnd)
plt.subplot(3,1,3)
plt.plot(forces['timestamp'].values, forces['zForce2'].values)
plt.plot(forces['timestamp'].values, forces['yForce2'].values)
plt.xlim(unixTAIStart, unixTAIEnd)

In [ ]:
for i in range(len(FATABLE)):
    print(FATABLE[i][M1M3FATable.FATABLE_ID], FATABLE[i][M1M3FATable.FATABLE_TYPE], FATABLE[i][M1M3FATable.FATABLE_ORIENTATION], \
         FATABLE[i][M1M3FATable.FATABLE_XINDEX], FATABLE[i][M1M3FATable.FATABLE_YINDEX], FATABLE[i][M1M3FATable.FATABLE_ZINDEX])


In [ ]:
FATABLE[28]

In [ ]:
thisBump = bumps[bumps['actuatorId']==102]

In [ ]:
thisBump['secondaryTest0']

In [ ]:
thisBump[thisBump['secondaryTest0']==2]

In [ ]:
thisBump[thisBump['primaryTest1']==2]['timestamp'].values[0]

In [ ]:
id = 229
thisBump = bumps[bumps['actuatorId']==id]
index = M1M3FATable.actuatorIDToIndex(id)
primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
    if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
        secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
        secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
        secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
else:
    secondaryName = None

print(primaryBump, primaryForce, secondaryBump, secondaryForce)
plt.figure(figsize=(10,5))
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
plotEnd = plotStart + 14.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
start = Time(plotStart, format='unix_tai', scale='tai')
end = Time(plotEnd, format='unix_tai', scale='tai')
forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
times = forces['timestamp'].values
t0 = times[0]
times -= t0
plotStart -= t0
plotEnd -= t0
plt.title(f"Primary - Z - ID:{id}")
plt.plot(times, forces[primaryForce].values)
if thisBump[primaryBump].values[-1] == 6:
    plt.text(2.0, 350.0, "PASSED", color='g')
elif thisBump[primaryBump].values[-1] == 7:
    plt.text(2.0, 350.0, "FAILED", color='r')

plt.xlim(plotStart, plotEnd)
plt.ylim(-400,400)
plt.xlabel("Time (seconds)")
plt.ylabel("Force (nt)")
plt.subplot(1,2,2)
if secondaryName is not None:
    plt.title(f"Secondary - {secondaryName} - ID:{id}")
    plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.plot(times, forces[secondaryForce].values)
    if thisBump[secondaryBump].values[-1] >= 5:
        plt.text(2.0, 350.0, "PASSED", color='g')
    elif thisBump[secondaryBump].values[-1] == 7:
        plt.text(2.0, 350.0, "FAILED", color='r')
    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
else:
    plt.title("No Secondary")
    plt.xticks([])
    plt.yticks([])


In [ ]:
thisBump

In [ ]:
thisBump['primaryTest1']

In [ ]:
M1M3FATable.actuatorIDToIndex(102)

In [ ]:
M1M3FATable.

In [ ]:
FATABLE[28][M1M3FATable.FATABLE_SINDEX]

In [ ]:
[M1M3FATable.FATABLE_ZFA]